In [1]:
%matplotlib notebook

In [2]:
import numpy as np
import pandas as pd
import gmaps
import requests
import json

# Google developer API key
from config import gkey

In [3]:
# read previous outputs into new dataframes
toronto_schools = pd.read_csv("output_data/toronto_schools.csv")
catholic_schools = pd.read_csv("output_data/toronto_catholic_schools.csv")

In [4]:
top_ts=toronto_schools.nlargest(10,"RATINGS")[["SCHOOL NAME","ADDRESS","FSA","RATINGS"]]
top_cs=catholic_schools.nlargest(10,"RATINGS")[["SCHOOL NAME","ADDRESS","FSA","RATINGS"]]

In [5]:
# Visualize top 10 Toronto Schools
top_ts

,SCHOOL NAME,ADDRESS,FSA,RATINGS
11,Fleming,20 Littles Road,M1B,9.5
166,Whitney,119 Rosedale Heights Drive,M4T,9.5
129,Denlow,50 Denlow Boulevard,M3B,9.3
164,John Ross Robertson,130 Glengrove Avenue West,M4R,9.3
245,Humber Valley Village,65 Hartfield Road,M9A,9.3
115,Beverly Glen,85 Beverly Glen Boulevard,M1W,9.2
163,Blythwood,2 Strathgowan Crescent,M4N,9.1
211,Ossington/Old Orchard,380 Ossington Avenue,M6J,9.1
233,Swansea,207 Windermere Avenue,M6S,9.1
88,Iroquois,265 Chartland Boulevard South,M1S,9.0


In [6]:
# Visualize top 10 Catholic Schools
top_cs

,SCHOOL NAME,ADDRESS,FSA,RATINGS
104,St Sebastian,717 Brock Avenue,M6H,10.0
84,St Michael's Choir,66 Bond St,M5B,8.8
132,Josyf Cardinal Slipyj,35 West Deane Park Drive,M9B,8.8
133,St Clement,4319 Bloor Street West,M9C,8.8
56,St Bonaventure,1340 Leslie Street,M3C,8.7
81,Our Lady of Perpetual Help,1 1/2 Garfield Avenue,M4T,8.6
141,St Demetrius,125 La Rose Avenue,M9P,8.4
35,Prince of Peace,255 Alton Towers Circle,M1V,8.3
40,St Henry,100 Bamburgh Circle,M1W,8.3
122,St Josaphat,110 Tenth Street,M8V,8.3


In [7]:
# read output house prices into new dataframe
house_price=pd.read_csv("output_data/toronto_api_postal_combined and filtered with price range 100K to 10M.csv")
house_price.head()

,Index,AreaName,Price ($),lat,lng,Address,City,Postal Code,FSA
0,19,Malvern,114900,43.803925,-79.216507,#105 - 8 Silverbell Grve,Toronto,M1B 2L7,M1B
1,46,Malvern,125000,43.805328,-79.220230,#310 - 99 BLACKWELL AVE,Toronto,M1B 3R5,M1B
2,47,Malvern,125000,43.805328,-79.220230,#101 - 99 BLACKWELL AVE,Toronto,M1B 3R5,M1B
3,74,Malvern,139900,43.805328,-79.220230,#405 - 99 BLACKWELL AVE,Toronto,M1B 3R5,M1B
4,79,Malvern,143000,43.803925,-79.216507,#504 - 8 SILVER BELL GRVE,Toronto,M1B 2L7,M1B


In [8]:
top_ts_list=list(top_ts["FSA"])
top_cs_list=list(top_cs["FSA"])

ts_house_price=house_price[house_price["FSA"].isin(top_ts_list)]
cs_house_price=house_price[house_price["FSA"].isin(top_cs_list)]

ts_house_price.head()

,Index,AreaName,Price ($),lat,lng,Address,City,Postal Code,FSA
0,19,Malvern,114900,43.803925,-79.216507,#105 - 8 Silverbell Grve,Toronto,M1B 2L7,M1B
1,46,Malvern,125000,43.805328,-79.220230,#310 - 99 BLACKWELL AVE,Toronto,M1B 3R5,M1B
2,47,Malvern,125000,43.805328,-79.220230,#101 - 99 BLACKWELL AVE,Toronto,M1B 3R5,M1B
3,74,Malvern,139900,43.805328,-79.220230,#405 - 99 BLACKWELL AVE,Toronto,M1B 3R5,M1B
4,79,Malvern,143000,43.803925,-79.216507,#504 - 8 SILVER BELL GRVE,Toronto,M1B 2L7,M1B


In [9]:
cs_house_price.head()

,Index,AreaName,Price ($),lat,lng,Address,City,Postal Code,FSA
729,16,Armdale,109900,43.820740,-79.261734,#218 - 3300 Mcnicoll Ave,Toronto,M1V 5J6,M1V
730,48,Armdale,125000,43.829632,-79.254021,#40 - 501 Passmore Ave,Toronto,M1V 5G4,M1V
731,49,Armdale,125000,43.829632,-79.254021,#40 - 501 Passmore Ave,Toronto,M1V 5G4,M1V
732,56,Agincourt,129000,43.806225,-79.289284,#1 - 3290 MIDLAND AVE,Toronto,M1V 3L3,M1V
733,57,Agincourt,129000,43.806137,-79.288139,#D106 - 3278 Midland Ave,Toronto,M1V 0C7,M1V


In [10]:
# find coordinates of top Toronto schools using reverse geocode by address
params = {"key": gkey,"language":"en"}
for index, row in top_ts.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    address = row["ADDRESS"]
    params['address'] = f"{address},+Toronto,+ON"
    ts_lat_lng = requests.get(base_url, params=params)
    ts_lat_lng = ts_lat_lng.json()
    top_ts.loc[index, "lat"] = ts_lat_lng["results"][0]["geometry"]["location"]["lat"]
    top_ts.loc[index, "lng"] = ts_lat_lng["results"][0]["geometry"]["location"]["lng"]
  
top_ts

,SCHOOL NAME,ADDRESS,FSA,RATINGS,lat,lng
11,Fleming,20 Littles Road,M1B,9.5,43.815572,-79.203430
166,Whitney,119 Rosedale Heights Drive,M4T,9.5,43.687169,-79.378400
129,Denlow,50 Denlow Boulevard,M3B,9.3,43.745682,-79.364153
164,John Ross Robertson,130 Glengrove Avenue West,M4R,9.3,43.718335,-79.408867
245,Humber Valley Village,65 Hartfield Road,M9A,9.3,43.665464,-79.526731
115,Beverly Glen,85 Beverly Glen Boulevard,M1W,9.2,43.798642,-79.323143
163,Blythwood,2 Strathgowan Crescent,M4N,9.1,43.719852,-79.389197
211,Ossington/Old Orchard,380 Ossington Avenue,M6J,9.1,43.652716,-79.422603
233,Swansea,207 Windermere Avenue,M6S,9.1,43.643466,-79.477088
88,Iroquois,265 Chartland Boulevard South,M1S,9.0,43.802681,-79.269392


In [11]:
# find coordinates of top Catholic schools using reverse geocode by address
params = {"key": gkey,"language":"en"}
for index, row in top_cs.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    address = row["ADDRESS"]
    params['address'] = f"{address},+Toronto,+ON"
    cs_lat_lng = requests.get(base_url, params=params)
    cs_lat_lng = cs_lat_lng.json()
    top_cs.loc[index, "lat"] = cs_lat_lng["results"][0]["geometry"]["location"]["lat"]
    top_cs.loc[index, "lng"] = cs_lat_lng["results"][0]["geometry"]["location"]["lng"]
    
top_cs

,SCHOOL NAME,ADDRESS,FSA,RATINGS,lat,lng
104,St Sebastian,717 Brock Avenue,M6H,10.0,43.662775,-79.439930
84,St Michael's Choir,66 Bond St,M5B,8.8,43.655122,-79.377964
132,Josyf Cardinal Slipyj,35 West Deane Park Drive,M9B,8.8,43.659593,-79.565700
133,St Clement,4319 Bloor Street West,M9C,8.8,43.632187,-79.573821
56,St Bonaventure,1340 Leslie Street,M3C,8.7,43.732907,-79.355948
81,Our Lady of Perpetual Help,1 1/2 Garfield Avenue,M4T,8.6,43.687710,-79.383622
141,St Demetrius,125 La Rose Avenue,M9P,8.4,43.684740,-79.521614
35,Prince of Peace,255 Alton Towers Circle,M1V,8.3,43.825423,-79.278853
40,St Henry,100 Bamburgh Circle,M1W,8.3,43.813806,-79.327984
122,St Josaphat,110 Tenth Street,M8V,8.3,43.599350,-79.509096


In [12]:
# Set coordinates for schools
coordinates_ts=top_ts[["lat", "lng"]]
coordinates_cs=top_cs[["lat","lng"]]

# Set coordinates for houses in Toronto school districts and Catholic school districs
coordinates_houses_ts=ts_house_price[["lat","lng"]]
coordinates_houses_cs=cs_house_price[["lat","lng"]]

In [13]:
# Access maps with unique API key
gmaps.configure(api_key=gkey)

# Customize the size of the figure
figure_layout = {'width': '1000px', 'height': '600px','border': '1px solid black','padding': '1px', 'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout)

# Assign the marker layers to ts and cs variables, with T for Toronto School and C for Catholic School
marker_ts=gmaps.marker_layer(coordinates_ts,label="T",info_box_content=[f"School: {school}" for school in top_ts["SCHOOL NAME"]])
marker_cs=gmaps.marker_layer(coordinates_cs,label="C",info_box_content=[f"School: {school}" for school in top_cs["SCHOOL NAME"]])

# Add the layer of Toronto and Catholic school markers to the map
fig.add_layer(marker_ts)
fig.add_layer(marker_cs)

# fig

In [14]:
# Create dataset for heat layers
price_ts=ts_house_price["Price ($)"]
price_cs=cs_house_price["Price ($)"]

# Set heat layer for house price in Toronto School district with weight as house price
heat_layer_ts = gmaps.heatmap_layer(coordinates_houses_ts,weights=price_ts,dissipating=False, max_intensity=10000000,point_radius = 0.01)
fig.add_layer(heat_layer_ts)

# Set heat layer for house price in Toronto School district with weight as house price
heat_layer_cs = gmaps.heatmap_layer(coordinates_houses_cs,weights=price_cs,dissipating=False, max_intensity=10000000,point_radius = 0.01)
fig.add_layer(heat_layer_cs)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [158]:
# # We kept the option to compare School places with all the housing data

# house_locations=house_price[["lat","lng"]]
# price=house_price["Price ($)"]

# # Create a poverty Heatmap layer using all housing data
# heat_layer = gmaps.heatmap_layer(house_locations,weights=price,dissipating=False, max_intensity=10000000,point_radius = 0.01)
# fig.add_layer(heat_layer)

# fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…